In [52]:
# Importing the required libraries
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning import Callback
import numpy as np
# Setting the random seed for reproducibility
random_seed = 42
torch.manual_seed(random_seed)

# Defining the batch size, available GPUs, and number of workers
BATCH_SIZE=128
AVAIL_GPUS = min(1, torch.cuda.device_count())
NUM_WORKERS=int(os.cpu_count() / 2)

This block of code imports various libraries required for building a deep learning model using PyTorch. It also sets the random seed to 42 for reproducibility of results.

Furthermore, it defines the batch size for the data loader, the number of available GPUs, and the number of workers for the data loader to use.
***

In [53]:
torch.cuda.is_available()

True

In [54]:
# Define CustomDataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.FFT_files = os.listdir(root_dir)
        self.transform = transform

    def __len__(self):
        return len(self.FFT_files)

    def __getitem__(self, idx):
        file_name=self.FFT_files[idx]
        FFT_path = os.path.join(self.root_dir, file_name)
        FFT = pd.read_csv(FFT_path,header=None).values
        FFT_tensor = torch.tensor(FFT, dtype=torch.float32)
        FFT_tensor = FFT_tensor.view(-1, FFT_tensor.shape[1])
        Label_Tag=file_name.split('_')
        label=[]
        for n in [1,3,5,7,9,11,13]:
            temp_num=float(Label_Tag[n])
            label.append(temp_num)
        label=torch.tensor(label)
    
        return FFT_tensor, label

In [55]:
## Converting the 2D data matrix to a 3D matrix

class CustomDataModule(pl.LightningDataModule):
    def __init__(self, csv_root, transform=None, batch_size=32, num_workers=0):
        super().__init__()
        self.csv_root = csv_root   # Directory path where the CSV file is stored
        self.batch_size = batch_size   # The batch size for the data loader
        self.num_workers = num_workers   # The number of worker processes for loading the data
        self.transform = transform   # Optional data transformation to be applied

    def prepare_data(self):
        pass   # Placeholder for any data preparation step, if needed

    def setup(self, stage=None):
        self.dataset = CustomDataset(self.csv_root, transform=self.transform)   # Initialize the CustomDataset class with the specified CSV directory and transform
        #self.dataset_train, self.dataset_val = random_split(self.dataset,[int(len(self.dataset)*0.7),len(self.dataset)-int(len(self.dataset)*0.7)])
        self.dataset_train = self.dataset   # Set the training dataset to be the entire dataset

    def train_dataloader(self):
        return DataLoader(self.dataset_train, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)   # Return a DataLoader object for the training data, which shuffles the data and divides it into batches

    def val_dataloader(self):
        return DataLoader(self.dataset_train, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)   # Return a DataLoader object for the validation data, which is set to be the same as the training data

    def test_dataloader(self):
        return None   # No test data is used for this model, so return None


In [56]:
# Setting the root directory of the dataset
root_dir = '../Data/FFT_Data/Turn_off'

# Defining the transformations for the dataset using PyTorch's Compose function
# transformations = transforms.Compose([
#     transforms.ToTensor(),
# ])

# Creating a custom dataset and dataloader using the CustomDataModule class
data_module = CustomDataModule(root_dir, transform=None)

In [57]:
# Set up the data module and data loader
data_module.setup()

# Retrieve the training data from the data loader
train_dataloader = data_module.train_dataloader()

# Get the first batch of data from the data loader
i, l = next(iter(train_dataloader))

In [58]:
# Printing the shape of the input data and labels
print(i.shape)
print(l.shape)

torch.Size([32, 3, 981])
torch.Size([32, 7])


### Discriminator



In [59]:
# A discriminator model that determines if an image is real or fake, outputting a single value between 0 and 1

## TODO: Change the Channel and input size

# Start [batch, 3, 981] using 1-D Convolution, and using dialation for using data diffenrent part data 
# [batch, 3, 981] > [batch, 3, 979] > [batch, 3, 975] > [batch, 3, 967] > [batch, 3, 951] > [batch, 3, 919] > [batch, 3, 855] > [batch, 3, 727] > [batch, 3, 471] <[batch, 1, 469] 
# linear [469, 200, 1]
#  
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        # CNN architecture with dialation  references by TCN
        self.conv1 = nn.Conv1d(3, 3, kernel_size=3,dilation=1)
        self.conv2 = nn.Conv1d(3, 3, kernel_size=3,dilation=2)
        self.conv3 = nn.Conv1d(3, 3, kernel_size=3,dilation=4)
        self.conv4 = nn.Conv1d(3, 3, kernel_size=3,dilation=8)
        self.conv5 = nn.Conv1d(3, 3, kernel_size=3,dilation=16)
        self.conv6 = nn.Conv1d(3, 3, kernel_size=3,dilation=32)
        self.conv7 = nn.Conv1d(3, 3, kernel_size=3,dilation=64)
        self.conv8 = nn.Conv1d(3, 3, kernel_size=3,dilation=128)
        self.conv9 = nn.Conv1d(3, 1, kernel_size=3)

        # FCN
        self.fc1 = nn.Linear(469, 200)
        self.fc2 = nn.Linear(200, 1)

        
  
    def forward(self, x):
        # Apply convolutional and ReLU activation
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = F.relu(self.conv8(x))
        x = F.relu(self.conv9(x))
        # Flatten the tensor so it can be fed into the fully connected layers
        temp_length=x.shape[2]
        x = x.view(-1, temp_length)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)

        # Apply sigmoid activation to output a value between 0 and 1
        return torch.sigmoid(x)

### Generator

In [60]:
## A generator model that takes a latent space vector as input and outputs a wavelet image

## TODO: 
# Input_size [batch,7]
# End_size [batch, 3, 981]

# size history [7] > [64] > [488] [3,490] > [3, 981]
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        # Define the layers of the generator
        self.lin1 = nn.Linear(7, 64)  
        self.lin2 = nn.Linear(64, 488)  
        self.ct1 = nn.ConvTranspose1d(1, 3, 3, stride=1) 
        self.ct2 = nn.ConvTranspose1d(3, 3, 3, stride=2) 
    

    def forward(self, x):
        # Pass the input through a linear layer and reshape
        batch_size=x.shape[0]
        x = self.lin1(x)
        x = F.relu(x)

        x = self.lin2(x)
        x = F.relu(x)

        x=x.view(batch_size,1,-1)

        # Upsample to  [batch, 3, 489]
        x = self.ct1(x)
        x = F.relu(x)
        
        # Upsample to [batch,3, 981] (16 feature maps)
        x = self.ct2(x)
        x = F.relu(x)
        

        return x

### GAN

In [61]:
## TODO: Change the input Data using the label

class GAN(pl.LightningModule):
    def __init__(self,Result_path,lr=0.0002):
        super().__init__()
        # Save the hyperparameters and initialize the generator and discriminator
        self.automatic_optimization=False
        self.save_hyperparameters()
        self.generator = Generator()
        self.discriminator = Discriminator()
        self.save_result_path=Result_path
        
    def forward(self, z):
        return self.generator(z)
    
    def adverarial_loss(self, y_hat,y):
        # Calculate binary cross-entropy loss
        return F.binary_cross_entropy(y_hat, y)
    
    def training_step(self, batch, batch_idx):
        g_opt, d_opt = self.optimizers()
        real_data, labels = batch
        self.for_result=labels
        # Sample noise
        z = labels
        z = z.type_as(real_data)
        
        # Train the generator: maximize log(D(G(z)))
        generate_data = self(z)
        y_hat = self.discriminator(generate_data)
        
        y = torch.ones(real_data.size(0), 1)
        y = y.type_as(real_data)
        
        g_loss = self.adverarial_loss(y_hat, y)
        
        g_opt.zero_grad()
        self.manual_backward(g_loss)
        g_opt.step()
        log_dict = {"g_loss" : g_loss }
    
        # Train the discriminator: maximize log(D(x)) + log(1 - D(G(z)))
        y_hat_real = self.discriminator(real_data)
        y_real = torch.ones(real_data.size(0), 1)
        y_real = y_real.type_as(real_data)
        real_loss = self.adverarial_loss(y_hat_real, y_real)
        
        y_hat_fake = self.discriminator(self(z))
        y_fake = torch.zeros(real_data.size(0), 1)
        y_fake = y_fake.type_as(real_data)
        fake_loss = self.adverarial_loss(y_hat_fake, y_fake)
        
        d_loss = (real_loss + fake_loss) / 2
        
        d_opt.zero_grad()
        self.manual_backward(d_loss)
        d_opt.step()
        log_dict = {"d_loss" : d_loss }
        return {"d_loss": d_loss,"g_loss":g_loss, "progress bar" : log_dict, "log": log_dict}
                
    def configure_optimizers(self):
        lr=self.hparams.lr
        opt_g = torch.optim.Adam(self.generator.parameters(), lr=lr)
        opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr)
        return opt_g, opt_d
    
    
    def on_train_end(self):
        z=self.for_result.type_as(self.generator.lin1.weight)
        z=self(z)
        Result=z.cpu().detach().numpy()
        batch=Result.shape[0]
        Result=Result.reshape(batch * 3 , -1)
        Result= pd.DataFrame(Result,index=None)
        Result.to_csv(self.save_result_path,header=False,index=False)

    def on_train_epoch_end(self):
        print('epoch',self.current_epoch)

In [62]:
# Setting the root directory of the dataset
root_dir = '../Data/FFT_Data/Turn_off'

# Creating a custom dataset and dataloader using the CustomDataModule class
data_module = CustomDataModule(root_dir, transform=None)
# Create an instance of the GAN model
save_result_path="../Data/Gan_Data/FFT/Result.csv"
model = GAN(Result_path=save_result_path)
# Set up the trainer object

trainer = pl.Trainer(max_epochs=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [63]:
trainer.fit(model,data_module)

c:\Users\SMEET_SIMUL\anaconda3\envs\torch\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
You are using a CUDA device ('NVIDIA GeForce RTX 3090 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 32.3 K
1 | discriminator | Discriminator | 94.5 K
------------------------------------------------
126 K     Trainable params
0         Non-traina

Epoch 0: 100%|██████████| 15/15 [00:07<00:00,  1.90it/s, v_num=17]

`Trainer.fit` stopped: `max_epochs=1` reached.


epoch 0
Epoch 0: 100%|██████████| 15/15 [00:07<00:00,  1.90it/s, v_num=17]
